# Predicting YouTube views: Large-language models

In [ ]:
#pip install --upgrade transformers

In [1]:
#!pip install tf-keras
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [2]:
import pandas as pd

samples = pd.read_csv('https://raw.githubusercontent.com/tommyliphysics/tommyli-ml/main/youtube_predictor/data/train.csv', lineterminator='\n')
samples

,channel_title,video_category,video_title,video_description,months,video_view_count,label
0,BKTVOK,22,BAWAR KHAN SON MUHAMMAD Khan Short video YouT...,Bawarkhan SON MUHAMMAD khan \nAmazing Videos\n...,19,1.982271,0
1,Rockit14,20,Add realistic waves to Minecraft! (Physics Mod),Play Minecraft with realistic physics! This mi...,11,5.338389,1
2,MAD ABOUT SCIENCE,22,The Stirling Engine at my Institute,Very close to Carnot Engine\n\n This one is...,50,5.685385,1
3,BKTVOK,22,Shergarh Bazar video YouTube amazing viralvide...,Shergarh Bazar video YouTube amazing viralvide...,21,2.287802,0
4,HVTraining,17,Cycling Tips: The science of electrolytes and ...,Looking for a proven training plan? \nhttps://...,72,3.307282,0
...,...,...,...,...,...,...,...
25323,Khanish,22,Friction welding #tools #science #viral,Friction welding is a solid-state welding proc...,57,3.766115,0
25324,TungaloyCorporation,28,We made a smile with high feed machining! #cn...,Product : AddDoFeed\nShank : VSSD08L090S05-C\n...,2,5.627098,1
25325,Sansad TV,25,Science Monitor | 14.08.2021,1.HUMAN-BASED MODELS TO STUDY NEURODEVELOPMENT...,32,4.421341,0
25326,The Truth Show,27,Trick for Reactivity Series of Metals #shorts ...,Join our Telegram Group ATP STAR JEE/NEET 2024...,18,6.748217,1


In [3]:
samples = samples.fillna('')

In [4]:
video_categories = {1: 'Film & Animation',
                    2: 'Autos & Vehicles',
                    10: 'Music',
                    15: 'Pets & Animals',
                    17: 'Sports',
                    19: 'Travel & Events',
                    20: 'Gaming',
                    22: 'People & Blogs',
                    23: 'Comedy',
                    24: 'Entertainment',
                    25: 'News & Politics',
                    26: 'Howto & Style',
                    27: 'Education',
                    28: 'Science & Technology',
                    29: 'Nonprofits & Activism'}

samples['video_category'] = samples['video_category'].apply(lambda category: video_categories[category])

In [5]:
val = samples.sample(frac=0.2, random_state=524)
train = samples.drop(val.index)

In [6]:
train

,channel_title,video_category,video_title,video_description,months,video_view_count,label
0,BKTVOK,People & Blogs,BAWAR KHAN SON MUHAMMAD Khan Short video YouT...,Bawarkhan SON MUHAMMAD khan \nAmazing Videos\n...,19,1.982271,0
1,Rockit14,Gaming,Add realistic waves to Minecraft! (Physics Mod),Play Minecraft with realistic physics! This mi...,11,5.338389,1
2,MAD ABOUT SCIENCE,People & Blogs,The Stirling Engine at my Institute,Very close to Carnot Engine\n\n This one is...,50,5.685385,1
3,BKTVOK,People & Blogs,Shergarh Bazar video YouTube amazing viralvide...,Shergarh Bazar video YouTube amazing viralvide...,21,2.287802,0
4,HVTraining,Sports,Cycling Tips: The science of electrolytes and ...,Looking for a proven training plan? \nhttps://...,72,3.307282,0
...,...,...,...,...,...,...,...
25323,Khanish,People & Blogs,Friction welding #tools #science #viral,Friction welding is a solid-state welding proc...,57,3.766115,0
25324,TungaloyCorporation,Science & Technology,We made a smile with high feed machining! #cn...,Product : AddDoFeed\nShank : VSSD08L090S05-C\n...,2,5.627098,1
25325,Sansad TV,News & Politics,Science Monitor | 14.08.2021,1.HUMAN-BASED MODELS TO STUDY NEURODEVELOPMENT...,32,4.421341,0
25326,The Truth Show,Education,Trick for Reactivity Series of Metals #shorts ...,Join our Telegram Group ATP STAR JEE/NEET 2024...,18,6.748217,1


In [7]:
val

,channel_title,video_category,video_title,video_description,months,video_view_count,label
11683,CrashCourse,Education,Micro-Biology: Crash Course History of Science...,It's all about the SUPER TINY in this episode ...,66,5.778978,1
14892,Padhle Tenthies,Education,Chemical Reactions and Equations Class 10 Scie...,Chemical Reactions and Equations Class 10 Scie...,23,5.384982,1
21540,Coding with Lewis,Science & Technology,What code editor should you use? 👩‍💻 #technolo...,,21,6.074111,1
4075,Hitesh Gohel,People & Blogs,Redox reaction class 11th Science gujarati,Balancing Redox reaction in acidic medium,60,4.482845,0
2171,MCQ Questions Hub,Education,Magnetism (चुम्बकत्व) MCQ | Physics MCQ | MCQ ...,Magnetism (चुम्बकत्व) MCQ | Physics MCQ | MCQ ...,35,3.995591,0
...,...,...,...,...,...,...,...
23360,Espiri Ibarra,People & Blogs,"October 9, 2022",,18,0.301030,0
11858,Happy Day,People & Blogs,Kirchhoff's Law,Kirchhoff’s Law-Pfis19-2B-Mohammad Nafis Nailu...,46,2.607455,0
12996,Book Scribe,Education,NCERT Science Class 6 தமிழ் | Chapter 13 | Fun...,NCERT Science Class 6 - Chapter 13: Fun with m...,27,3.859559,0
7290,OBD Việt Nam - Dịch vụ TỪ XA,Autos & Vehicles,Introduction SCR (Selective catalytic reductio...,#obdvietnam #service \n-----------------------...,1,1.568202,0


We will now define functions to import the pre-trained model from the huggingface transformers library.

In [8]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
from transformers import DistilBertTokenizerFast

def get_tokenizer_model():
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
    model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)
    return tokenizer,model

In [9]:
tokenizer,model = get_tokenizer_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreT

We will use keras to fine-tune the pre-trained models on our human and AI-generated texts. Let's define a function that prepares text samples for training, and then prepare the training and validation data.

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

sep_token = tokenizer.convert_tokens_to_ids('[SEP]')
cls_token = tokenizer.convert_tokens_to_ids('[CLS]')

def create_dataset(samples):
    encodings = {col: tokenizer(samples[col].tolist(), add_special_tokens=False)['input_ids'] for col in ['channel_title', 'video_title', 'video_description', 'video_category']}
    encodings = [[cls_token] + channel_title + [sep_token] + video_title + [sep_token] + video_description + [sep_token] + video_category + [sep_token]
                  for channel_title, video_title, video_description, video_category
                  in zip(encodings['channel_title'], encodings['video_title'], encodings['video_description'], encodings['video_category'])]
    encodings = pad_sequences(encodings, maxlen=tokenizer.model_max_length, padding='post', truncating='post', value=tokenizer.pad_token_id)

    return tf.data.Dataset.from_tensor_slices((encodings,samples['label'].tolist())).shuffle(len(samples)).batch(16)

In [13]:
train_dataset = create_dataset(train)
val_dataset = create_dataset(val)

Next we can define our functions to compile and train the model. I'll create a custom callback that saves the model to a specified directory.

In [17]:
import numpy as np

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler

import math

import keras
from keras.callbacks import Callback

last_epoch = 0

def compile_model(model):
    model.compile(optimizer=RMSprop(learning_rate=learning_rate),
                  metrics = ['accuracy'])
    model.config.id2label = {0: 'human', 1: 'AI'}

def fit_model(model):
    history = model.fit(train_dataset,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[LearningRateScheduler(lr_scheduler),
                   EarlyStopping(monitor="val_loss", patience=3)],
        validation_data=val_dataset,
        verbose=1)



We can now compile and fine tune the keras model. I'll start training at a learning rate of 10$^{-5}$ then reduce it by a factor of 10 for 3 more epochs.

In [15]:
learning_rate = 1e-5
compile_model(model)

In [16]:
def lr_scheduler(epoch, lr):
    return learning_rate*(0.5**epoch)

In [18]:
epochs = 20
batch_size=128

In [19]:
fit_model(model)

Epoch 1/20


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1267/1267 [==============================] - 1355s 1s/step - loss: 0.6008 - accuracy: 0.6637 - val_loss: 0.5492 - val_accuracy: 0.7185 - lr: 1.0000e-05
Epoch 2/20
1267/1267 [==============================] - 1311s 1s/step - loss: 0.5203 - accuracy: 0.7421 - val_loss: 0.5088 - val_accuracy: 0.7574 - lr: 5.0000e-06
Epoch 3/20
1267/1267 [==============================] - 1308s 1s/step - loss: 0.4788 - accuracy: 0.7728 - val_loss: 0.5005 - val_accuracy: 0.7580 - lr: 2.5000e-06
Epoch 4/20
1267/1267 [==============================] - 1310s 1s/step - loss: 0.4523 - accuracy: 0.7922 - val_loss: 0.5083 - val_accuracy: 0.7604 - lr: 1.2500e-06
Epoch 5/20
1267/1267 [==============================] - 1311s 1s/step - loss: 0.4373 - accuracy: 0.7988 - val_loss: 0.4946 - val_accuracy: 0.7623 - lr: 6.2500e-07
Epoch 6/20
1267/1267 [==============================] - 13

In [20]:
def lr_scheduler(epoch, lr):
    return 1e-7

In [21]:
fit_model(model)

Epoch 1/20
1267/1267 [==============================] - 1298s 1s/step - loss: 0.4220 - accuracy: 0.8090 - val_loss: 0.4929 - val_accuracy: 0.7610 - lr: 1.0000e-07
Epoch 2/20
1267/1267 [==============================] - 1309s 1s/step - loss: 0.4197 - accuracy: 0.8108 - val_loss: 0.4927 - val_accuracy: 0.7606 - lr: 1.0000e-07
Epoch 3/20
1267/1267 [==============================] - 1311s 1s/step - loss: 0.4217 - accuracy: 0.8101 - val_loss: 0.4926 - val_accuracy: 0.7625 - lr: 1.0000e-07
Epoch 4/20
1267/1267 [==============================] - 1308s 1s/step - loss: 0.4198 - accuracy: 0.8101 - val_loss: 0.4921 - val_accuracy: 0.7621 - lr: 1.0000e-07
Epoch 5/20
1267/1267 [==============================] - 1307s 1s/step - loss: 0.4182 - accuracy: 0.8131 - val_loss: 0.4916 - val_accuracy: 0.7623 - lr: 1.0000e-07
Epoch 6/20
1267/1267 [==============================] - 1309s 1s/step - loss: 0.4180 - accuracy: 0.8140 - val_loss: 0.4913 - val_accuracy: 0.7619 - lr: 1.0000e-07
Epoch 7/20
1267/1267 [

In [16]:
fit_model(model)

1267/1267 [==============================] - 1233s 973ms/step - loss: 0.5374 - accuracy: 0.7302 - val_loss: 0.5448 - val_accuracy: 0.7235 - lr: 1.0000e-05


In [17]:
fit_model(model)

1267/1267 [==============================] - 1232s 972ms/step - loss: 0.4880 - accuracy: 0.7669 - val_loss: 0.5207 - val_accuracy: 0.7527 - lr: 1.0000e-05


In [18]:
fit_model(model)

1267/1267 [==============================] - 1222s 965ms/step - loss: 0.4377 - accuracy: 0.7967 - val_loss: 0.5551 - val_accuracy: 0.7430 - lr: 1.0000e-05


In [19]:
fit_model(model)

1267/1267 [==============================] - 1222s 965ms/step - loss: 0.3812 - accuracy: 0.8254 - val_loss: 0.6175 - val_accuracy: 0.7294 - lr: 1.0000e-05


In [20]:
fit_model(model)

1267/1267 [==============================] - 1229s 970ms/step - loss: 0.3266 - accuracy: 0.8548 - val_loss: 0.5972 - val_accuracy: 0.7531 - lr: 1.0000e-05


In [21]:
fit_model(model)

1267/1267 [==============================] - 1233s 973ms/step - loss: 0.2821 - accuracy: 0.8745 - val_loss: 0.6864 - val_accuracy: 0.7487 - lr: 1.0000e-05


In [22]:
fit_model(model)

1267/1267 [==============================] - 1234s 974ms/step - loss: 0.2435 - accuracy: 0.8930 - val_loss: 0.7275 - val_accuracy: 0.7410 - lr: 1.0000e-05


In [23]:
fit_model(model)

1267/1267 [==============================] - 1233s 973ms/step - loss: 0.2163 - accuracy: 0.9038 - val_loss: 0.9025 - val_accuracy: 0.7475 - lr: 1.0000e-05


In [24]:
fit_model(model)

1267/1267 [==============================] - 1233s 973ms/step - loss: 0.2008 - accuracy: 0.9105 - val_loss: 0.9068 - val_accuracy: 0.7444 - lr: 1.0000e-05


In [25]:
fit_model(model)

1267/1267 [==============================] - 1234s 974ms/step - loss: 0.1884 - accuracy: 0.9159 - val_loss: 0.9449 - val_accuracy: 0.7477 - lr: 1.0000e-05


In [26]:
fit_model(model)

1267/1267 [==============================] - ETA: 0s - loss: 0.1751 - accuracy: 0.9212

KeyboardInterrupt: 

In [ ]:
fit_model(model)

In [ ]:
fit_model(model)

In [ ]:
fit_model(model)

In [ ]:
fit_model(model)

In [ ]:
fit_model(model)

In [ ]:
fit_model(model)

In [ ]:
fit_model(model)

In [ ]:
fit_model(model)

In [ ]:
learning_rate = 1e-6

In [ ]:
fit_model(model)

956/956 [==============================] - 946s 990ms/step - loss: 0.0122 - accuracy: 0.9967 - val_loss: 0.0193 - val_accuracy: 0.9953 - lr: 1.0000e-06


In [ ]:
learning_rate = 1e-7

In [ ]:
fit_model(model)

956/956 [==============================] - 946s 990ms/step - loss: 0.0097 - accuracy: 0.9974 - val_loss: 0.0220 - val_accuracy: 0.9940 - lr: 1.0000e-07


In [ ]:
learning_rate = 1e-8

In [ ]:
fit_model(model)

956/956 [==============================] - 945s 989ms/step - loss: 0.0091 - accuracy: 0.9979 - val_loss: 0.0220 - val_accuracy: 0.9940 - lr: 1.0000e-08


In the last epoch, the training loss slightly decreased, but there was no decrease in the validation loss. I'll continue training for an additional epoch at the same learning rate.

In [ ]:
fit_model(model)

956/956 [==============================] - 947s 991ms/step - loss: 0.0090 - accuracy: 0.9980 - val_loss: 0.0220 - val_accuracy: 0.9940 - lr: 1.0000e-08


We can see that the training loss has only improved by 1% and the validation loss has not improved at all. I'll slightly increase the learning rate in the next epoch and see if there is an improvement.

In [ ]:
learning_rate = 2e-8
fit_model(model)

956/956 [==============================] - 950s 994ms/step - loss: 0.0090 - accuracy: 0.9978 - val_loss: 0.0220 - val_accuracy: 0.9940 - lr: 2.0000e-08


We again see that there is no improvement, so I'll increase the learning rate again:

In [ ]:
learning_rate = 5e-8
fit_model(model)

956/956 [==============================] - 946s 990ms/step - loss: 0.0089 - accuracy: 0.9981 - val_loss: 0.0222 - val_accuracy: 0.9935 - lr: 5.0000e-08


We see that the training loss has decreased slightly while the validation loss has increased. It appears we are now beginning to overfit. To confirm this, I'll train for another epoch with the same learning rate:

In [ ]:
fit_model(model)

956/956 [==============================] - 947s 990ms/step - loss: 0.0091 - accuracy: 0.9980 - val_loss: 0.0224 - val_accuracy: 0.9935 - lr: 5.0000e-08


It seems that we have found the optimal model, so we will use the learning rate scheduling we found to train a new model on the full training set.

In [ ]:
train = samples[samples['TTV split'] >= 0]
val = samples[samples['TTV split'] == 0]
train_dataset = create_dataset(train)

train_dataset = create_dataset(train)
val_dataset = create_dataset(val)

In [ ]:
learning_rates = [1e-5, 1e-6, 1e-7]

def lr_scheduler(epoch, lr):
    return learning_rates[epoch]

In [ ]:
learning_rate = 1e-5
compile_model(model)

In [ ]:
epochs = 3
batch_size=128
fit_model(model)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1716984207.814080     116 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1195/1195 [==============================] - 1254s 1s/step - loss: 0.0921 - accuracy: 0.9652 - val_loss: 0.0141 - val_accuracy: 0.9963 - lr: 1.0000e-05
Epoch 2/3
1195/1195 [==============================] - 1196s 1s/step - loss: 0.0098 - accuracy: 0.9974 - val_loss: 0.0178 - val_accuracy: 0.9956 - lr: 1.0000e-06
Epoch 3/3
1195/1195 [==============================] - 1194s 999ms/step - loss: 0.0079 - accuracy: 0.9983 - val_loss: 0.0156 - val_accuracy: 0.9958 - lr: 1.0000e-07


We can now push our trained model to the huggingface hub:

In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()
#model.push_to_hub('ai-detector-distilbert')
#push_to_hub('ai-detector-distilbert')

The model is now available on the huggingface hub at https://huggingface.co/tommyliphys/ai-detector-distilbert.